<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_calendar_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a calendar in PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

In [15]:
pip install borb

We're now going to build a helper method that will construct a `Table` to represent the grid of the calendar.

The precise layout may require some fiddling if you want to change it.

Essentially, we need to find out how many incomplete weeks there will be (maybe the month starts/ends with some days before hitting Monday). And we need to take into account the headers of the `Table` (which will be the names of the days of the week) and some cells that will just be empty padding.

In [16]:
from borb.pdf.canvas.layout.table.fixed_column_width_table import FixedColumnWidthTable
from borb.pdf.canvas.layout.table.table import Table, TableCell
from calendar import monthrange
import typing
from decimal import Decimal
from borb.pdf.canvas.layout.layout_element import Alignment

def build_table_for_month(month: int, year: int) -> FixedColumnWidthTable:

  number_of_days: int = monthrange(year, month)[1]
  first_day_of_month: int = monthrange(year, month)[0]
  
  weekdays_per_day: typing.List[int] = [(x + first_day_of_month) % 7 for x in range(0, number_of_days)]
  
  number_of_weeks: int = sum([1 for x in weekdays_per_day if x == 6])
  if weekdays_per_day[-1] != 6:
    number_of_weeks += 1

  t: Table = FixedColumnWidthTable(number_of_columns=7, number_of_rows=number_of_weeks + 2)

  # build header
  for s in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
    t.add(Paragraph(s, 
                    font="Helvetica-Bold", 
                    font_size=Decimal(8),
                    horizontal_alignment=Alignment.CENTERED,
                    padding_bottom=Decimal(5),
                    padding_top=Decimal(5)
    ))

  # these cells serve as spacing
  for i in range(0, 7):
    t.add(TableCell(Paragraph(" "), 
                    border_top=False,
                    border_right=False,
                    border_bottom=False,
                    border_left=False))  

  # add empty days (the month may not start on a Monday)
  for i in range(0, first_day_of_month):
    t.add(TableCell(Paragraph(" "), 
                    border_top=False,
                    border_right=False,
                    border_bottom=False,
                    border_left=False))

  # add days for month
  for i in range(1, number_of_days + 1):
    t.add(TableCell(Paragraph(str(i),
                    horizontal_alignment=Alignment.RIGHT,
                    padding_top=Decimal(20),
                    padding_bottom=Decimal(5),
                    padding_right=Decimal(5)),
                    border_top=True,
                    border_right=True,
                    border_bottom=True,
                    border_left=True,
                    )
    )

  # add empty days
  for i in range(weekdays_per_day[-1], 6):
    t.add(TableCell(Paragraph(" "), 
                    border_top=False,
                    border_right=False,
                    border_bottom=False,
                    border_left=False))
  
  return t


That was the hard part!

Now we just need to call that function 12 times, and spruce up the remainder of the `Page` a bit (by adding the month name and a picture).

In [17]:
from borb.pdf.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.layout.image.image import Image


# Font
from borb.pdf.canvas.font.simple_font.true_type_font import TrueTypeFont  
from borb.pdf.canvas.font.font import Font
from pathlib import Path  

# create new Document
doc: Document = Document()

months: typing.List[typing.Tuple[str, str]] = [("January", "https://images.unsplash.com/photo-1491831947735-7f519f52f6db?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1373&q=80"),
                                               ("February", "https://images.unsplash.com/photo-1484979045040-0ab3854b6acb?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1448&q=80"),
                                               ("March", "https://images.unsplash.com/photo-1462275646964-a0e3386b89fa?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1528&q=80"),
                                               ("April", "https://images.unsplash.com/photo-1551524612-4b158646bc05?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1545&q=80"),
                                               ("May", "https://images.unsplash.com/photo-1614179741597-c24532a179a4?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1473&q=80"),
                                               ("June", "https://images.unsplash.com/photo-1473496169904-658ba7c44d8a?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1470&q=80"),
                                               ("July", "https://images.unsplash.com/photo-1520116468816-95b69f847357?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=687&q=80"),
                                               ("August", "https://images.unsplash.com/uploads/14121010130570e22bcdf/e1730efe?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1470&q=80"),
                                               ("September", "https://images.unsplash.com/photo-1600675206532-b0a6dbf01252?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=687&q=80"),
                                               ("October", "https://images.unsplash.com/photo-1509622905150-fa66d3906e09?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=735&q=80"),
                                               ("November", "https://images.unsplash.com/photo-1445855743215-296f71d4b49c?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=2070&q=80"),
                                               ("December", "https://images.unsplash.com/photo-1609226536789-dbc1b85ae9db?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1471&q=80")]

# add months
for month, (month_name, image_url) in enumerate(months):

  # create new Page
  page: Page = Page()
  doc.append_page(page)

  # set PageLayout
  layout: PageLayout = SingleColumnLayout(page)
  
  # add month name
  layout.add(Paragraph(month_name, 
                       font=TrueTypeFont.true_type_font_from_file(Path("IndieFlower-Regular.ttf")),
                       font_size=Decimal(20)))

  # add Image
  layout.add(Image(image_url, 
                   width=Decimal(470), 
                   height=Decimal(300)))

  # add Table
  layout.add(build_table_for_month(month + 1, 2022))

Now let's store our `Document`.

In [18]:
# store
with open("output_001.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, doc)

We've used a lot of (large) images; let's see how big our file is. The following code prints the size of our PDF in Mb.

In [19]:
import os
os.path.getsize("output_001.pdf") / 10**6

2.67747

Most of this is because PDF (like HTML) differentiates between the size of the image it uses, and the size at which that image is displayed. So, even though our images are displayed at fairly small resolution, they're probably stored (inside the document) at quite substantial size(s).

Luckily, `borb` comes with `ImageFormatOptimization`, which runs over the entire `Document` and checks the maximum display-size of each `Image`. It then changes the resolution of those `Image` instances to match.

For our `Document`, you can expect quite a dramatic improvement.

In [20]:
from borb.toolkit.image.image_format_optimization import ImageFormatOptimization

doc_002: typing.Optional[Document] = None
l: ImageFormatOptimization = ImageFormatOptimization()
with open("output_001.pdf", "rb") as pdf_file_handle:
  doc = PDF.loads(pdf_file_handle, [l])

assert doc is not None

# store PDF
with open("output_002.pdf", "wb") as out_file_handle:
  PDF.dumps(out_file_handle, doc)

Let's see what that gained us;

In [21]:
import os
os.path.getsize("output_002.pdf") / 10**6

0.414164